In [1]:
#todo: 
#move Jr to AF edit
#abstract remove long

In [1]:
import glob, os
from bs4 import BeautifulSoup
import re
import pandas as pd
pd.options.display.max_rows = 2000
import collections

In [3]:
#remove Jr,
articles=[]
folder="htmls_jmis_org"
files=os.listdir(folder)
for f in files:
    article={}
    html=open(folder+'/'+f).read()
    s = BeautifulSoup(html, 'html.parser')
    authors=s.find_all(href=re.compile("contributors"))
    al=[a.text.replace(' Jr,',',') for a in authors]
    authors=[(a.split(',')[1]+' '+a.split(',')[0]).strip() for a in al]
    authors='; '.join(authors)
    keywords=s.find_all(href=re.compile("keywords"))
    keywords=[k.text.strip() for k in keywords]
    keywords='; '.join(keywords)
    title=s.find_all('h3')[0].text
    issue=s.find_all(href=re.compile("issues"))[0].text
    issuel=issue.split('Volume ')[1].split(' Number ')
    VL=issuel[0]
    IS=issuel[1][0:1]
    PY=issuel[1][2:6]
    pages=s.find_all(text=re.compile('pp\.'))[0].replace(' pp. ','')
    cont=s.find_all('div',{'class':'container'})[0]
    abstract=cont.findChildren(recursive=False)[4].text.replace('ABSTRACT: ','').replace('\n','')
    #if len(abstract)>0:
    #    abstract=abstract[0].replace('ABSTRACT: ','').replace('\n','')
    #else:
    #    abstract=''
    article['DE']=keywords
    article['AF']=authors
    article['TI']=title
    article['VL']=VL
    article['IS']=IS
    article['PY']=PY
    article['PG']=pages
    article['AB']=abstract
    article['SO']='Journal of Management Information Systems'
    article['UR']='http://www.jmis-web.org/articles/'+f
    articles.append(article)

In [4]:
df=pd.DataFrame(articles)
df.PG.fillna('-',inplace=True)
df['BP']=df['PG'].apply(lambda x: x.split('-')[0])
df['EP']=df['PG'].apply(lambda x: x.split('-')[1])

In [5]:
df=df[df.TI!='Editorial Introduction']
df=df[df.TI!='About This Issue']
df=df[df.TI!='From the Editor-in-Chief']
df=df[~df.TI.str.contains('Special Issue:')]
df=df[~df.TI.str.contains('Special Section:')]

In [6]:
#remove ()
DE=[]
for index, row in df.iterrows():
    keywords=row['DE'].split(';')
    keywords_=''
    for keyword in keywords:
        keyword=re.sub(r'\(.+?\)', '', keyword).strip()
        words=keyword.split()
        words_=''
        for idx, word in enumerate(words):
            if idx>0:
                if len(word)>1:
                    if (word[0].isupper() and word[1].islower()):
                        word=word.lower()
            words_=words_+' '+word
        if keywords_=='':
            keywords_=words_        
        else:
            keywords_=keywords_+'; '+words_   
    DE.append(keywords_)
print len(DE)
a=[]
for row in DE:
    a=a+(row.split(';'))
a=[i.strip() for i in a]
counter=collections.Counter(a)
counter.most_common(10)


1137


[(u'decision support systems', 57),
 (u'group support systems', 46),
 (u'electronic commerce', 36),
 (u'knowledge management', 33),
 (u'trust', 27),
 (u'group decision support systems', 27),
 (u'expert systems', 25),
 (u'outsourcing', 20),
 (u'end-user computing', 19),
 ('', 17)]

In [7]:
#Add . 
AF=[]
for index, row in df.iterrows():
    authors=row['AF'].split(';')
    authors_=''
    for author in authors:
        words=author.split()
        words_=''
        for idx, word in enumerate(words):
            if len(word)==1:
                word=word+'.'
            words_=words_+' '+word
        if authors_=='':
            authors_=words_        
        else:
            authors_=authors_+'; '+words_   
    AF.append(authors_)
print len(AF)
a=[]
for row in AF:
    a=a+(row.split(';'))
a=[i.strip() for i in a]
counter=collections.Counter(a)
counter.most_common(10)

1137


[(u'Jay F. Nunamaker', 46),
 (u'Eric K. Clemons', 29),
 (u'Robert J. Kauffman', 21),
 (u'Andrew B. Whinston', 21),
 (u'Varun Grover', 17),
 (u'Alan R. Dennis', 17),
 (u'Robert O. Briggs', 16),
 (u'Izak Benbasat', 15),
 (u'Paul Benjamin Lowry', 11),
 (u'Abraham Seidmann', 10)]

In [8]:
df['DE']=DE
df['AF']=AF

In [9]:
rem='''\[ABSTRACT FROM PUBLISHER\]Copyright of Journal of Management Information Systems is the property of M.E. Sharpe Inc. and its content may not be copied or emailed to multiple sites or posted to a listserv without the copyright holder\'s express written permission. However, users may print, download, or email articles for individual use. This abstract may be abridged. No warranty is given about the accuracy of the copy. Users should refer to the original published version of the material for the full abstract. \(Copyright applies to all Abstracts.\)'''
df['AB']=df.AB.str.replace(rem,'')

In [10]:
rep='\xc3\xa0' #visavis
df['TI']=df.TI.str.replace(rep,'A')

In [11]:
df.to_csv('jmis_1984_2016_dec.csv',columns=['AF','TI','DE','AB','SO','VL','PY','IS','BP','EP'],encoding='utf-8')

In [162]:
#work in progress
articles=[]
folder="htmls"
files=os.listdir(folder)[304:308]
l=[]
for f in files:
    if 'html' in f:
        print f
        article={}
        html=open(folder+'/'+f).read()
        s = BeautifulSoup(html, 'html.parser')
        authors=s.find_all('div', {'class':'addAuthorInfo'})
        for author in authors:
            a=author.findChild().findChild()
            f=a.findChild().text
            l=a.text.replace(f,'')
            aff=author.text.replace(f+l,'').replace(';','')
            
            
            


10.1080_07421222.1993.11517996.html
10.1080_07421222.1993.11517997.html
10.1080_07421222.1993.11517998.html
10.1080_07421222.1993.11517999.html


In [159]:
aff

u' is Assistant Professor of Management at the MIT Sloan School of Management. He focuses his research on the economic impacts of infonnation technology. He is currently analyzing how organizations can restructure decision making, compensation systems, and ownership structure to benefit from advances in infonnation technology. He is also interested in understanding the productivity of infonnation technology investments. Professor Brynjolfsson perfonned his doctoral work at MIT in managerial economics and received bachelor\u2019s and master\u2019s degrees from Harvard in applied mathematics.'

In [161]:
f

u'Erik'